<h1>
importing important libraries


In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import nltk

In [2]:
df= pd.read_csv('spam.csv', encoding='ISO-8859-1')
df.head()
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [3]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
df.head()
df = df.rename(columns={'v1':'target', 'v2':'text'})

In [4]:
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


<h2>Changing to binary

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])
# df.drop(columns=['num_letters'], inplace=True)
df.head(5)

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.isnull().sum()

target    0
text      0
dtype: int64

<h2>removing stopwords from text

In [7]:
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
X=[]
# c=0
# for i in range(len(df['text'])):
#     c+=1
#     print(df['text'][i])
#     if(c==5) : break

print(type(df['text'][0]))

for i in range(len(df['text'])):
    # for j in df['text'][i]:
        review = re.sub('[^a-zA-Z0-9]', ' ', df['text'][i])
        review = review.lower()
        review = review.split()
        review = [stemmer.stem(word) for word in review  if not word in set(stopwords.words('english'))]
        review = ' '.join(review)
        X.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Priyansh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'str'>


In [8]:
X[:3]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18']

<h2> now using BOW to create them into vectors


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cv= CountVectorizer(binary=True)
X_cv = cv.fit_transform(X)

pickle.dump(cv,open('CountVectorizer.pkl', 'wb'))

print(X_cv[0].toarray())


[[0 0 0 ... 0 0 0]]


<h2> now using TFIDF to create vectors

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(2,3))
X_tfidf = tfidf.fit_transform(X)
print(X_tfidf[0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [15]:
y=df['target']

<h2> creating function for model training

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
def train_model(X, y,model, modelName):
    X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('Accuracy of {}  is : {} %'.format(modelName,accuracy_score(y_pred,y_test)*100))
    print('Precision of {}  is : {} %'.format(modelName,precision_score(y_pred,y_test)*100))
    return model

<h3> traing using multiple naive byes

In [17]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
model1 = train_model(X_cv,y, nb, 'Multinomial Naive Byes')
# model1 = train_model(X_tfidf,y, nb, 'Multinomial Naive Byes')
# model1 = train_model(X_tfidf,y, nb, 'Multinomial Naive Byes')

Accuracy of Multinomial Naive Byes  is : 97.66816143497758 %
Precision of Multinomial Naive Byes  is : 92.0 %


In [18]:
from sklearn.naive_bayes import BernoulliNB
gnb = BernoulliNB()
model3 = train_model(X_cv,y,gnb, 'Bernauli NB')

Accuracy of Bernauli NB  is : 97.75784753363229 %
Precision of Bernauli NB  is : 84.66666666666667 %


<p> since accuracy is very high(97%) this shows overfitting of data

<h3> training using logistic regression

In [19]:
x=["Congratulations! You have been selected to receive an exclusive $1,000 gift card from Amazon. This limited-time offer is available only to a select few, so don't miss out on this amazing opportunity."]
x= cv.transform(x)
x.toarray()
ans=model1.predict(x)
ans

array([1])

In [20]:
from sklearn.linear_model import LogisticRegression
lgr= LogisticRegression()
model2=train_model(X_cv,y,lgr, 'Logistic Regression')
# model2=train_model(X_tfidf,y,lgr, 'Logistic Regression')

Accuracy of Logistic Regression  is : 97.66816143497758 %
Precision of Logistic Regression  is : 82.66666666666667 %


<h3> now saving these model using pickle to save time fro loading next time

In [21]:
import pickle
fileName1= 'MultinomialNB.pkl'
fileName2= 'LogisticRegression.pkl'
pickle.dump(model1,open(fileName1,'wb'))
pickle.dump(model2,open(fileName2,'wb'))

<h2> Now main work begin to get input from user and predict the output

In [1]:
def model_prediction(model, sentence):
    # conversion to BOW
    sen = cv.transform(sentence)
    ans = model.predict(sen)
    return ans

In [5]:
import pickle

sentence =["""
Hi Team,

I hope you all are doing well. As a reminder, we have our weekly team meeting scheduled for tomorrow at 10:00 AM in the main conference room.

The agenda for the meeting includes project updates where each team member will provide a brief update on their current projects and discuss any challenges or roadblocks. We will also review feedback received from our recent client meetings and plan actionable steps based on the feedback.

Additionally, we will review upcoming project deadlines to ensure we are on track and assign any additional tasks as necessary. We will also discuss ideas for our next team building activity. Finally, we will open the floor for any questions or additional points anyone wants to discuss.

Please come prepared with your updates and any points you would like to discuss. If you have any items to add to the agenda, let me know by the end of today.

Looking forward to seeing you all there!
           
           """]
cv = pickle.load(open('CountVectorizer.pkl','rb'))
model = pickle.load(open('MultinomialNB.pkl', 'rb'))
prediction = model_prediction(model, sentence)
prediction

array([0])